In [1]:
# 필요 시 주석을 해제하고 아래 라이브러리들을 설치
# !pip install pystan==2.19.1.1
# !pip3 install fbprophet
# !pip install plotly

In [2]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet

c:\Users\WAI\Anaconda3\envs\AIcontest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 데이터 로드
data = pd.read_csv('./preprocessed_data.csv', index_col=0)
data['ds'] = data.index
train = json.load(open('./train.json', 'r', encoding='utf8')) 
submit = json.load(open('./sample_submission.json', 'r', encoding='utf8')) 

In [1]:
# 사업장별 dataframe 제작
# df_dic = {}
plc_lst = list(train.keys())
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

# for i in plc_lst:
#     plc_df = pd.DataFrame(columns={'ds', 'pH', 'COD', 'SS', 'N', 'P', 'T'})
#     plc_df = plc_df[['ds', 'pH', 'COD', 'SS', 'N', 'P', 'T']]
    
#     date_lst = list(train[i].keys())
#     plc_df.loc[:, 'ds'] = date_lst

#     for index, j in enumerate(date_lst):
#         for k in fct_lst:
#             try:
#                 plc_df.loc[index, k] = train[i][j][k]
#             except:
#                 pass
                    
#     df_dic[i] = plc_df

NameError: name 'train' is not defined

In [3]:
plc_lst[:5]

NameError: name 'plc_lst' is not defined

In [6]:
# df_dic[plc_lst[0]]

In [7]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
future

,ds,y
0,2018-02-01,NaN
1,2018-02-02,NaN
2,2018-02-03,NaN
3,2018-02-04,NaN
4,2018-02-05,NaN
...,...,...
694,2019-12-27,NaN
695,2019-12-28,NaN
696,2019-12-29,NaN
697,2019-12-30,NaN


In [8]:
# 추론
start_time = time.time()

for i in plc_lst:
    plc_df = data[data['local']==i]
    
    for j in fct_lst:
        fct_df = plc_df[['ds', j]]
        fct_df = fct_df.rename(columns={j:'y'})
        model = Prophet(growth='flat')
        model.fit(fct_df)
        forecast = model.predict(future)
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast = forecast[['ds', 'yhat']]
        
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
c:\Users\WAI\Anaconda3\envs\AIcontest\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
c:\Users\WAI\Anaconda3\envs\AIcontest\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
c:\Users\WAI\Anaconda3\envs\AIcontest\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\WAI\AppData\Local\Temp\ipykernel_14504\2059595312.py:17: DeprecationWarning: `np.float` is a deprecated al

In [ ]:
# 추론 파일 저장
with open('./prediction0616.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)